In [ ]:
!pip install cloudscraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 4.0 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 


Функция для очищения (на всякий случай) от htlm-тегов.

In [ ]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

Тут собираем на странице автора его ник, потому что на странице с выдачей почему-то нельзя его вытащить.

In [ ]:
def get_author(link):
  scraper = cloudscraper.create_scraper()
  response = scraper.get(link)
  soup = BeautifulSoup(response.text, "html.parser")

  name = soup.find('div', {'class': 'user-name'})
  return name.text

Собираем ссылки на тексты + названия + ник автора и ссылку на его страницу.

In [ ]:
def get_urls(link, id_ff):
  urls = []

  scraper = cloudscraper.create_scraper()
  response = scraper.get(link)
  soup = BeautifulSoup(response.text, "html.parser")
  articles = soup.find_all('div', {'class': 'js-toggle-description'})
  for article in articles:
    title = article.find('a', {'class': 'visit-link'})
    ff_link = 'https://ficbook.net' + title.attrs['href']
    title = title.text
    author = article.find('span', {'class': 'author word-break'})
    author = author.find('a')
    author_link = 'https://ficbook.net' + author.attrs['href']
    author = get_author(author_link)
    urls.append([id_ff, title, ff_link, author, author_link])
    id_ff += 1
  return urls

Собираем текст каждого фанфика.

In [ ]:
def get_text(link):
  scraper = cloudscraper.create_scraper()
  response = scraper.get(link)
  soup = BeautifulSoup(response.text, "html.parser")

  text = soup.find('div', {'id': 'content'})
  return cleanhtml(text.text)

In [ ]:
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
import cloudscraper

In [ ]:
import csv

In [ ]:
session = requests.session()

Записываем всю информацию в .tsv файл. Каждому фанфику присваиваем уникальный id, записываем метаинформацию о нем, а так же текст.

Смотрите продолжение в файлел **project_parse_ffs.ipynb**.

In [ ]:
cnt_added = 0
p = 2
with open('ffs.tsv', 'w', encoding='utf-8') as f:
  writer = csv.writer(f, delimiter='\t')
  writer.writerow(['id_ff', 'title', 'ff_link', 'author', 'author_link', 'text'])
  while cnt_added < 30:
    link = f"https://ficbook.net/find?fandom_filter=any&fandom_group_id=1&pages_range=6&pages_min=&pages_max=1&statuses%5B0%5D=2&ratings%5B0%5D=5&ratings%5B1%5D=6&ratings%5B2%5D=7&transl=1&tags_exclude%5B0%5D=1692&likes_min=&likes_max=&rewards_min=&date_create_min=2022-09-22&date_create_max=2022-10-22&date_update_min=2022-09-22&date_update_max=2022-10-22&title=&sort=1&rnd=994345839&find=%D0%9D%D0%B0%D0%B9%D1%82%D0%B8!&p={p}#result"
    ffs = get_urls(link, cnt_added)
    for ff in ffs:
      text = get_text(ff[2])
      ff.append(text)
      writer.writerow(ff)
      cnt_added += 1
    p += 1
